In [ ]:
%%capture

! pip install datasets
! pip install transformers
! pip install accelerate -U
! pip install evaluate

In [ ]:
!unzip archive.zip # данные с https://www.kaggle.com/datasets/mikhailma/russian-social-media-text-classification/data

Archive:  archive.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import torch
import gc

import numpy as np
import pandas as pd

from datasets import Dataset, DatasetDict # api hf Dataset
from transformers import AutoTokenizer # токенизатор
from transformers import DataCollatorWithPadding # collator -- нужно для обучения на торч
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer # fine-tuning api

import evaluate # библиотека метрик

In [ ]:
# загрузим данные через пандас (потому что в нем некоторые вещи анализируются проще и быстрее, чем в hf Dataset)

train_data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [ ]:
train_data

,oid,category,text
0,365271984,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...
1,503385563,extreme,Возвращение в подземелье Треша 33 Эйфория тупо...
2,146016084,football,Лучшие чешские вратари – Доминик Доминатор Гаш...
3,933865449,boardgames,Rtokenoid Warhammer40k валрак решил нас подкор...
4,713550145,hockey,Шестеркин затаскивает Рейнджерс в финал Восточ...
...,...,...,...
38735,910636962,autosport,8 битная буря снова накрыла пикселями автомоби...
38736,669736851,autosport,Ира Сидоркова объясняет как сказалась на ее ма...
38737,558919241,tennis,24 я ракетка мира хорват Марин Чилич обыграл и...
38738,776944963,volleyball,Стал известен календарь мужской сборной России...


In [ ]:
test

,oid,text
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...
1,452466036,Естественное восстановление после тяжелой трен...
2,161038103,Тема нарядов продолжается Одна из британских ж...
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...
...,...,...
26255,169728316,Выиграй коллекционный пазл по Wortokenoid of W...
26256,279369911,Волейбол от первого лица Егора Пупынина переко...
26257,600699419,Вы были когда нибудь на свидании где вам задав...
26258,560223506,ТОП 20 самых эффективных общефизических упражн...


In [ ]:
sample_submission

,oid,category
0,1622114,athletics
1,1663817,autosport
2,3174332,basketball
3,3469228,extreme
4,3905302,boardgames
...,...,...
2621,998309713,esport
2622,998565619,esport
2623,999112505,boardgames
2624,999361308,tennis


# Data Transformation

In [ ]:
# объединим test c sample (чтобы получился полноценный тестовый датасет)
test_data = test.merge(sample_submission, on='oid')
test_data

,oid,text,category
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...,esport
1,749208109,Пусть это побудет здесь БорьбаВпрямомЭфире How...,esport
2,749208109,Раздача пиздюлей от Мунсунга. HowToFtokenoid Б...,esport
3,749208109,Не знаю как вам но мне стилистика нравится пус...,esport
4,749208109,Мне кажется тут каким то селфцестом пахнет. . ...,esport
...,...,...,...
26255,952958325,Arszeeqq Mag и Storm обсудили провал Atokenoid...,esport
26256,952958325,НС про дизбанд B8 Неужели все и бутылка шампан...,esport
26257,952958325,Юбилейный десятый сезон Wtokenoid Dota 2 Champ...,esport
26258,952958325,Rosttokenoid после взрыва петарды Меня милиция...,esport


In [ ]:
# теперь в обоих данных уберем колонку oid

train_data = train_data.iloc[:, 1:].copy()
test_data = test_data.iloc[:, 1:].copy()

In [ ]:
# далее проверим наши данные на пропущенные значения

train_data.isnull().sum()

category    0
text        0
dtype: int64

In [ ]:
test_data.isnull().sum()

text        0
category    0
dtype: int64

In [ ]:
# теперь проверим дубликаты и сразу же удалим

for name, df in [("train", train_data), ("test",test_data)]:
  d_count = df['text'].duplicated().sum()
  print(f'{name} df has {d_count} duplicates')
  if d_count > 0:
    print('removing duplicates...')
    df.drop_duplicates(subset=['text'], inplace=True)

train df has 2966 duplicates
removing duplicates...
test df has 1427 duplicates
removing duplicates...


In [ ]:
# теперь нам нужно векторизовать данные
# начнем с создания словаря целевой переменной

id2label = {n:name for n, name in enumerate(train_data['category'].unique())}
label2id = {name:n for n, name in id2label.items()}
label2id

{'winter_sport': 0,
 'extreme': 1,
 'football': 2,
 'boardgames': 3,
 'hockey': 4,
 'esport': 5,
 'athletics': 6,
 'motosport': 7,
 'basketball': 8,
 'tennis': 9,
 'autosport': 10,
 'martial_arts': 11,
 'volleyball': 12}

In [ ]:
# чтобы нам было легче векторизовывать, воспользуемся api hf

train_hf = Dataset.from_pandas(train_data, preserve_index=False)
test_hf = Dataset.from_pandas(test_data, preserve_index=False)

data = DatasetDict()
data['train'] = train_hf
data['validation'] = test_hf
data

DatasetDict({
    train: Dataset({
        features: ['category', 'text'],
        num_rows: 35774
    })
    validation: Dataset({
        features: ['text', 'category'],
        num_rows: 24833
    })
})

In [ ]:
model_name = 'ai-forever/ruBert-base' # наша модель, которую мы будем fine-tunin'ить
# я выбрал именно ее по двум причинам: 1) она уже обучена для русского языка;
# 2) bert учитывает контекст при векторизации, что снимает омонимию
tokenizer = AutoTokenizer.from_pretrained(model_name)

# напишем функцию для векторизации
def vectorize(batch):
  tokenized_inputs = tokenizer(batch['text'], truncation=True, max_length=512)
  tokenized_inputs['labels'] = [label2id[v] for v in batch['category']]
  return tokenized_inputs

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

In [ ]:
# собственно векторизация
vect_data = data.map(
            vectorize,
            batched=True,
            batch_size=8,
            remove_columns=data['train'].column_names # у данных колонки одинаковые, поэтому укажем любой
            )
vect_data

Map:   0%|          | 0/35774 [00:00<?, ? examples/s]

Map:   0%|          | 0/24833 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 35774
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 24833
    })
})

# Training

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# напишем метрику для оценки качества модели
accuracy = evaluate.load("accuracy") # из страницы kaggle просили использовать простую метрику точности

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
# скачаем модель для нашей задачи -- классификация текста

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(label2id.keys()), id2label=id2label, label2id=label2id
)

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# очищаем кэш, чтобы избежать ошибок с памятью
gc.collect()
torch.cuda.empty_cache()

# укажем параметры обучения
training_args = TrainingArguments(
    output_dir="text_classification_model_for_vk_groups",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

# запустим само обучение
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=vect_data["train"],
    eval_dataset=vect_data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train() # у нас получилось достичь 83% точности на одной эпохе (если увеличить, то и точность думаю станет лучше)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.590300,0.609091,0.832159


TrainOutput(global_step=2236, training_loss=0.8010006940428814, metrics={'train_runtime': 2690.201, 'train_samples_per_second': 13.298, 'train_steps_per_second': 0.831, 'total_flos': 6036626866940928.0, 'train_loss': 0.8010006940428814, 'epoch': 1.0})

In [ ]:
# сохраним модель

model_check = "./vk_classifier"
trainer.save_model(model_check)

# Usage a.k.a Inference

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model=model_check) # посмотрим как применяется наша модель

In [ ]:
classifier("Я пойду играть в футбол") # в целом верно

[{'label': 'football', 'score': 0.8191999793052673}]

In [ ]:
classifier("Турнир по шахматам")

[{'label': 'boardgames', 'score': 0.9224109649658203}]